In [1]:
import pandas as pd
import numpy as np

In [33]:
def CDS_code_maker(CC, DC, SC):
    if len(str(int(CC)))<2:
        CC = '0'+ str(int(CC))
    else:
        CC = str(int(CC))
    DC = str(int(DC))
    if len(str(int(SC)))<7:
        SC = '0'*(7-len(str(int(SC))))+ str(int(SC))
    else:
        SC = str(int(SC))
    if len(CC+DC+SC) != 14:
        print(CC+DC+SC)
    return CC+DC+SC

In [34]:
path = 'CA_edu_data/'

In [153]:
numeric = ['CohortStudents', 'RegularHSGradCount',
       'RegularHSGradRate', 'MetUCCSUCount', 'MetUCCSURate',
       'SealOfBiliteracyCount', 'SealOfBiliteracyRate',
       'GoldenStateSealMeritCount', 'GoldenStateSealMeritRate',
       'CHSPECompleterCount', 'CHSPECompleterRate', 'AdultEdDiplomaCount',
       'AdultEdDiplomaRate', 'SPEDCertificateCount', 'SPEDCertificateRate',
       'GEDCompleterCount', 'GEDCompleterRate', 'OtherTransferCount',
       'OtherTransferRate', 'DropoutCount', 'DropoutRate',
       'StillEnrolledCount', 'StillEnrolledRate']

In [204]:
grad_rates = pd.read_table(path+'grad_rates_cohort1617.txt', sep = '\t')
grad_rates = grad_rates[(grad_rates.AggLevel == 'S') & (grad_rates.DASSYN == 'No')& (grad_rates.CharterYN != 'All')]
grad_rates['CDS'] = [CDS_code_maker(grad_rates.CountyCode.iloc[ik], grad_rates.DistrictCode.iloc[ik], grad_rates.SchoolCode.iloc[ik]) for ik in range(len(grad_rates))]
grad_rates.reset_index()
grad_rates.set_index('CDS')

for col in numeric:
    try: 
        grad_rates[col] = grad_rates[col].str.replace("*", "0")
    except:
        print('no dice')
    grad_rates[col] = pd.to_numeric(grad_rates[col])

In [165]:
schools = grad_rates.groupby('CDS')
repcats = grad_rates.groupby('ReportingCategory')

In [166]:
repcat_cols = ['CohortStudents', 'RegularHSGradCount','RegularHSGradRate', 'MetUCCSUCount', 'MetUCCSURate','DropoutCount', 'DropoutRate']
meta_cols = ['CountyName', 'DistrictName', 'SchoolName','CDS', 'CharterYN']

In [167]:
grad_rates2 = grad_rates[['CDS', 'CharterYN']].drop_duplicates()

for repcat in repcats:
    sub_df = repcat[1][['CDS']]
    sub_df.set_index(['CDS'])
    for col in repcat_cols:
        sub_df[col+ '_' + repcat[0]] = repcat[1][col]
    grad_rates2 = grad_rates2.join(sub_df.set_index(['CDS'], verify_integrity=True), on = ['CDS'], how = 'outer')


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [168]:
def split_GS(grades_vect):

    def splitter(offered):
        try:
            [min_grade, max_grade] = offered.split('-')
        except:
            [min_grade, max_grade] = [np.nan, np.nan]
        return min_grade, max_grade

    min_grdes = []
    max_grdes = []
    for ik in range(len(grades_vect)):
        min_grade, max_grade = splitter(grades_vect[ik])
        min_grdes.append(min_grade)
        max_grdes.append(max_grade)
        
    return min_grdes, max_grdes

In [169]:
school_meta = pd.read_table(path + 'pubschls.txt', sep = '\t')
school_meta = school_meta.rename(columns={'CDSCode':'CDS'})# = school_meta
school_meta.reset_index()
school_meta.set_index('CDS')

,NCESDist,NCESSchool,StatusType,County,District,School,Street,StreetAbr,City,Zip,...,AdmFName1,AdmLName1,AdmEmail1,AdmFName2,AdmLName2,AdmEmail2,AdmFName3,AdmLName3,AdmEmail3,LastUpDate
CDS,,,,,,,,,,,,,,,,,,,,,
1100170000000,691051.0,NaN,Active,Alameda,Alameda County Office of Education,NaN,313 West Winton Avenue,313 West Winton Ave.,Hayward,94544-1136,...,L Karen,Monroe,lkmonroe@acoe.org,NaN,NaN,NaN,NaN,NaN,NaN,02/02/2017
1100170109835,691051.0,10546.0,Closed,Alameda,Alameda County Office of Education,FAME Public Charter,"39899 Balentine Drive, Suite 335","39899 Balentine Dr., Ste. 335",Newark,94560-5359,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/01/2015
1100170112607,691051.0,10947.0,Active,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,1515 Webster Street,1515 Webster St.,Oakland,94612-3355,...,Eve,Gordon,eve@envisionacademy.org,NaN,NaN,NaN,NaN,NaN,NaN,07/26/2017
1100170118489,691051.0,12283.0,Closed,Alameda,Alameda County Office of Education,Aspire California College Preparatory Academy,2125 Jefferson Avenue,2125 Jefferson Ave.,Berkeley,94703-1414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07/01/2015
1100170123968,691051.0,12844.0,Active,Alameda,Alameda County Office of Education,Community School for Creative Education,2111 International Boulevard,2111 International Blvd.,Oakland,94606-4903,...,Monique,Brinson,moniqueb@communityschoolforcreativeeducation.org,NaN,NaN,NaN,NaN,NaN,NaN,08/16/2017
1100170124172,691051.0,12901.0,Active,Alameda,Alameda County Office of Education,Yu Ming Charter,1086 Alcatraz Avenue,1086 Alcatraz Ave.,Oakland,94608-1265,...,Sue,Park,spark@yumingschool.org,NaN,NaN,NaN,NaN,NaN,NaN,05/10/2016
1100170125567,691051.0,13008.0,Active,Alameda,Alameda County Office of Education,Urban Montessori Charter,5328 Brann Street,5328 Brann St.,Oakland,94619-3312,...,Jen,Heeter,jen@urbanmontessori.org,NaN,NaN,NaN,NaN,NaN,NaN,04/10/2018
1100170130302,691051.0,NaN,Closed,Alameda,Alameda County Office of Education,"Technical, Agricultural & Nat.",597 C Street,597 C St.,Hayward,94541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/24/1999
1100170130401,691051.0,9264.0,Active,Alameda,Alameda County Office of Education,Alameda County Juvenile Hall/Court,2500 Fairmont Avenue,2500 Fairmont Ave.,San Leandro,94578-1005,...,Sean,Moffatt,smoffatt@acoe.org,NaN,NaN,NaN,NaN,NaN,NaN,07/03/2018


In [170]:
min_grades, max_grades = split_GS(school_meta.GSoffered)
school_meta['GSoffered_min'] = min_grades
school_meta['GSoffered_max'] = max_grades

min_grades, max_grades = split_GS(school_meta.GSserved)
school_meta['GSserved_min'] = min_grades
school_meta['GSserved_max'] = max_grades

In [171]:
HS = school_meta[(school_meta.EdOpsCode == 'TRAD') & (school_meta.EILCode == 'HS')]
HS.reset_index()
HS = HS.set_index(['CDS'])

In [172]:
grad_rates2.reset_index()
grad_rates2 = grad_rates2.set_index(['CDS'])

In [173]:
HS = HS.join(grad_rates2)

In [183]:
EL = pd.read_table(path+ 'eng_learn1617.txt', sep = '\t')
EL.reset_index()
EL = EL.set_index('CDS')

In [203]:
lang_cols = ['GR_9', 'GR_10', 'GR_11', 'GR_12']
langs = EL.groupby('LANGUAGE')

for lang in langs:
    sub_df = repcat[1][['CDS']]
    sub_df.reset_index()
    sub_df = sub_df.set_index(['CDS'])
    for col in lang_cols:
        sub_df[col+ '_' + lang[0]] = lang[1][col]
    print(sub_df.index)
    grad_rates2 = grad_rates2.join(sub_df.set_index(['CDS'], verify_integrity=True), on = ['CDS'], how = 'outer')


Index(['01100170112607', '01611190130609', '01611190130229', '01611190106401',
       '01611190130625', '01611190132142', '01611190119222', '01611190000001',
       '01611270130450', '01611270000001',
       ...
       '57727025738505', '57727100000001', '57727100101162', '57727105738802',
       '58727286115935', '58727360000000', '58727365830013', '58727365830138',
       '58727365835202', '58727695838305'],
      dtype='object', name='CDS', length=1869)


KeyError: 'CDS'